# Detect particles in BSE images and EBSD intensity maps

Håkon Wiik Ånes (hakon.w.anes@ntnu.no)

In [7]:
# Switch to interactive Matplotlib backend (e.g. qt5) for control point determination
%matplotlib qt5

from datetime import date
import importlib_metadata
import os

from mapregions import MapRegions
import matplotlib.pyplot as plt
import numpy as np
from scipy import ndimage
import skimage.color as skc
import skimage.filters as skf
from skimage.segmentation import watershed


# Directories
sample = "0s"
dset_no = 2
dir_data = f"/home/hakon/phd/data/p/prover/{sample}/{dset_no}"
dir_bse = os.path.join(dir_data, "bse")
dir_imreg = os.path.join(dir_data, "imreg")
dir_kp = os.path.join(dir_data, "kp")
dir_partdet = os.path.join(dir_data, "partdet")

# Matplotlib
plt.rcParams.update({"axes.grid": False, "figure.figsize": (15, 5), "font.size": 12})
savefig_kw = dict(bbox_inches="tight", pad_inches=0, transparent=True, dpi=150)
scatter_kw = dict(s=70, linewidth=2, facecolor="none", clip_on=False)

print("Run date: ", date.today())
print("\nSoftware versions\n-----------------")
for pkg in ["mapregions", "matplotlib", "numpy", "orix", "scipy", "scikit-image"]:
    if pkg == "numpy":
        ver = np.__version__
    else:
        ver = importlib_metadata.version(pkg)
    print(pkg, ":", ver)

Run date:  2022-08-02

Software versions
-----------------
mapregions : 0.1.dev0
matplotlib : 3.5.2
numpy : 1.22.4
orix : 0.9.0.post0
scipy : 1.8.1
scikit-image : 0.19.3


Step sizes (`bse` is upscaled)

In [8]:
step_size = dict(ebsd=0.1, bse=0.025, bse_full=1 / 39.2)

Dataset specific parameters

In [16]:
# Remove zero-regions resulting from stitching of BSE images
crop_bse = {
    "0s": {
        1: (slice(0, -1), slice(0, -1)),
        2: (slice(0, -1), slice(0, -1)),
        3: (slice(0, -1), slice(0, -1)),
    },
    "175c": {
        1: (slice(107, -1), slice(273, -1)),
        2: (slice(117, -1), slice(0, -1)),
        3: (slice(224, 3670), slice(251, -1)),
    },
    "300c": {
        1: (slice(0, 3826), slice(0, -1)),
        2: (slice(0, 3418), slice(0, -1)),
        3: (slice(16, 3916), slice(0, -1)),
    },
    "325c": {
        1: (slice(0, -1), slice(0, -1)),
        2: (slice(0, 3277), slice(0, -1)),
        3: (slice(129, 3745), slice(0, -1)),
    },
}
crop_bse_full = {
    "0s": {
        1: (slice(28, 3795), slice(43, 5634)),
        2: (slice(36, 5973), slice(171, 5946)),
        3: (slice(10, 5560), slice(313, 5717)),
    },
    "175c": {
        1: (slice(147, 3962), slice(36, 4442)),
        2: (slice(None, None), slice(None, None)),
        3: (slice(None, None), slice(None, None)),
    },
    "300c": {
        1: (slice(1,  3749), slice(124, 5576)),
        2: (slice(16, 3837), slice(137, 5798)),
        3: (slice(25, 3949), slice(50,  5289))
    },
    "325c": {
        1: (slice(2, 3795), slice(570, 5850)),
        2: (slice(28, 3811), slice(17, 5497)),
        3: (slice(27, 3575), slice(199, 5786)),
    },
}

Image to load

In [10]:
#data_type = "ebsd"
#data_type = "bse"
data_type = "bse_full"

Load image

In [17]:
if data_type == "ebsd":
    mask = np.load(os.path.join(dir_imreg, "mask_ebsd_correct.npy"))
    img = plt.imread(os.path.join(dir_imreg, "ebsd_correct.png"))
else:
    if data_type == "bse":
        img = plt.imread(os.path.join(dir_imreg, "bse_rescaled.png"))[crop_bse[sample][dset_no]]
    else:  # bse_full
        img = plt.imread(os.path.join(dir_bse, "4500x_cropped2_fused.png"))[crop_bse_full[sample][dset_no]]
    if len(img.shape) > 2:
        img = skc.rgb2gray(img[..., :3])
    mask = np.ones(img.shape, dtype=bool)

In [18]:
plt.figure()
plt.imshow(img)

## Detection steps

### 1. Intensity processing

In [7]:
# Flat field by subtracting long-range intensity variations
sigma = int(5 / step_size["bse"])  # microns / step size
blur = ndimage.gaussian_filter(img, sigma=sigma)
img2 = img - blur

In [8]:
# Normalize to enable thresholding independent of absolute intensities
img3 = img2 - img2[mask].mean()
img3 = img3 / np.linalg.norm(img3[mask])

In [9]:
figsize = (15, 5 * img.shape[0] / img.shape[1])

# Inspect intensity processing
fig, ax = plt.subplots(ncols=3, sharex=True, sharey=True, figsize=figsize)
ax[0].imshow(img, cmap="gray")
ax[1].imshow(blur)
ax[2].imshow(img3, cmap="gray")
for a in ax:
    a.axis("off")
fig.tight_layout(pad=0.5)
fig.savefig(os.path.join(dir_partdet, f"{data_type}_maps_intensity_processing.png"), **savefig_kw)

### 2. Generate elevation map

In [10]:
# Elevation map
elevation_map = skf.sobel(img3)

### 3. Determine markers for background and particles

In [11]:
int_range = (img3[mask].min(), img3[mask].max())
diff = abs(np.diff(int_range)[0])

# Threshold for markers
threshold = skf.threshold_triangle(img3[mask])
background = threshold + 0.15 * diff
particles = threshold + 0.25 * diff

# Intensity histogram
fig, ax = plt.subplots()
ax.hist(img3[mask].ravel(), bins=255, range=int_range);
ax.set_xlabel("Intensity")
ax.set_ylabel("Frequency")
ax.set_xlim(int_range)
ax.axvspan(int_range[0], background, color="C1", alpha=0.5, label="Background")
ax.axvspan(particles, int_range[1], color="C2", alpha=0.5, label="Particles")
ax.legend()
fig.savefig(os.path.join(dir_partdet, f"{data_type}_hist_intensity.png"), **savefig_kw)

### 4. Compute watershed and 5. remove holes within particles segmented from the watershed

In [12]:
# Generate sure markers of background and particles from the extreme parts of the
# intensity histogram
markers = np.zeros_like(img3)
markers[img3 < background] = 1
markers[img3 > particles] = 2

# Compute the watershed transformation (computationally intensive)
segmentation = watershed(elevation_map, markers)

# Remove small holes with mathematical morphology
segmentation = ndimage.binary_fill_holes(segmentation - 1)

# Label particles
labeled_particles, n = ndimage.label(segmentation)
print(n)

# Inspect segmentation (useful to plot when optimizing processing and detection
# parameters)
fig, ax = plt.subplots(ncols=3, sharex=True, sharey=True, figsize=figsize)
ax[0].imshow(img3, cmap="gray")
ax[1].imshow(markers)
ax[2].imshow(img3, cmap="gray")
ax[2].contour(segmentation, colors="r", linewidths=0.5)
for a in ax:
    a.axis("off")
fig.tight_layout(pad=0.5)
fig.savefig(os.path.join(dir_partdet, f"{data_type}_maps_segmentation.png"), **savefig_kw)

3472


### 6. Remove incorrectly detected particles

In [13]:
# Remove incorrectly detected particles
regions = MapRegions(
    label_map=labeled_particles,
    background_label=0,
    intensity_image=img3,
)

mean_intensity = regions.mean_intensity
roundness = regions.roundness
solidity = regions.solidity

In [14]:
# Check spatial distribution of rejection properties
titles = ["Mean intensity", "Roundness", "Solidity"]
fig, ax = plt.subplots(ncols=3, sharex=True, sharey=True, figsize=figsize)
ax[0].imshow(regions.get_map_data(mean_intensity))
ax[1].imshow(regions.get_map_data(roundness))
ax[2].imshow(regions.get_map_data(solidity))
for i, ax_i in enumerate(ax):
    ax_i.set_title(titles[i])
    ax_i.axis("off")
fig.tight_layout(pad=0.5)
fig.savefig(os.path.join(dir_partdet, f"{data_type}_maps_reject.png"), **savefig_kw)

In [15]:
# Filter detected particles
threshold_mean_intensity = np.percentile(mean_intensity, 55)  # 55
threshold_roundness = np.percentile(roundness[roundness > 0], 20)  # 25
threshold_solidity = np.percentile(solidity[solidity > 0], 15)  # 25
print(threshold_mean_intensity, threshold_roundness, threshold_solidity)

keep = (
    mean_intensity > threshold_mean_intensity,
    roundness == 0,
    roundness > threshold_roundness,
    solidity == 0,
    solidity > threshold_solidity,
)

# Reject some particles
regions2 = regions[np.logical_or.reduce(keep)]
print(regions2)

0.0007200949476100504 0.9505779571638973 0.9907688811883999
MapRegions: 3466


In [16]:
# Check histograms of rejection properties
titles = ["Mean intensity", "Roundness", "Solidity"]
# Inspect thresholds
fig, (ax0, ax1, ax2) = plt.subplots(ncols=3, figsize=(9, 3))
ax0.hist(mean_intensity, bins=100);
ax1.hist(roundness, bins=100);
ax2.hist(solidity, bins=100);
ax0.axvspan(threshold_mean_intensity, mean_intensity.max(), color="C1", alpha=0.5, label="Keep")
ax1.axvspan(threshold_roundness, roundness.max(), color="C1", alpha=0.5, label="Keep")
ax2.axvspan(threshold_solidity, solidity.max(), color="C1", alpha=0.5, label="Keep")
for i, ax in enumerate((ax0, ax1, ax2)):
    ax.set_xlabel(titles[i])
    ax.set_ylabel("Frequency")
    ax.legend()
fig.tight_layout()
fig.savefig(os.path.join(dir_partdet, f"{data_type}_hist_reject.png"), **savefig_kw)

In [17]:
# Check plot of rejected particles
with plt.rc_context(rc={"lines.linewidth": 0.5}):
    contour_kwds = dict(zorder=2)
    fig, ax = plt.subplots(sharex=True, sharey=True, figsize=(10, 10))
    ax.imshow(regions.intensity_image, cmap="gray")
    ax.contour(~regions.is_background_map, colors="red", **contour_kwds)
    ax.contour(~regions2.is_background_map, colors="lime", **contour_kwds)
    ax.axis("off")
    fig.tight_layout()
    fig.savefig(os.path.join(dir_partdet, f"{data_type}_maps_particles.png"), **savefig_kw)

In [18]:
# Save labeled particles to file
if data_type == "ebsd":
    np.save(os.path.join(dir_partdet, "ebsd_labels_filled"), labeled_particles)
elif data_type == "bse":
    np.save(os.path.join(dir_partdet, "bse_labels_filled_filtered"), regions2.label_map)
else:  # bse_full
    np.save(os.path.join(dir_partdet, "bse_full_labels_filled_filtered"), regions2.label_map)